<a href="https://colab.research.google.com/github/amun83/Koscom-Financial-Data-Bootcamp/blob/master/FD104_Oilprice_com_News_Text_Mining_and_One_More_Thing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting News Articles from https://oilprice.com/

In [0]:
ON_COLAB  = True
DATA_PATH = ''
MAX_PAGES = 1000

In [0]:
!pip install -q --upgrade pandas_datareader
!pip install -q --upgrade finance-datareader
!pip install -q --upgrade html5lib
!pip install -q --upgrade lxml
!pip install -q --upgrade seaborn
!pip install -q --upgrade gspread

In [0]:
# utilities
import datetime
import re
import csv
from IPython.display import display

# web scraping
import requests
import urllib.parse
from bs4 import BeautifulSoup as bs
import pandas_datareader as pdr
import FinanceDataReader as fdr

# Math/Stat
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
#import math

# Machine Learning
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import LSTM

# visualization
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
if ON_COLAB :
  from google.colab import drive
  drive.mount('/gdrive')
  DATA_PATH = '/gdrive/My Drive/oilprice.com/'

In [0]:
pattern = re.compile(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d\d),\s(\d{4})')
# Example: By Julianne Geiger - Dec 14, 2018, 8:00 PM CST

In [0]:
news_urls = []

for i in range(1, MAX_PAGES):
  page_url = 'https://oilprice.com/Latest-Energy-News/World-News/Page-' + str(i) + '.html'
  # https://oilprice.com/Latest-Energy-News/World-News/Page-865.html
  
  try:
    r = requests.get(page_url)
    print('Requested(GET): ' + page_url)
  except:
    print('Request(GET) Failed: ' + page_url)
    break

  soup = bs(r.text)
  items = soup.find_all('div', {"class":"categoryArticle__content"})

  for item in items:
    news_urls.append(item.find('a', href = True)['href'])
    #print('News URL Parsed: ' + news_urls[-1])

In [0]:
r = requests.get('https://oilprice.com/Latest-Energy-News/World-News/Page-866.html')
r

In [0]:
with open(DATA_PATH + 'oilprice.com_news.csv', 'w') as f:
  fw = csv.writer(f) 
  
  for url in news_urls:
    try:
      r = requests.get(url)
      print('Requested(GET): ' + url)
    except:
      print('Request(GET) Failed: ' + url)
      break

    soup = bs(r.text)

    title = soup.find('h1').text

    date = soup.find('span', {'class':'article_byline'}).text

    match = re.search(pattern, date)
    mmm, dd, yyyy = match.groups(0)
    date = yyyy + "-" + mmm + "-" + dd

    raw_content = soup.find('div', {'id':'news-content'}).find_all('p')
    content = ' '.join([p.text.strip() for p in raw_content[:-2]])

    #print('- Title:   ' + title)
    #print('- Date:    ' + date)
    #print('- Content: ' + content)
    
    fw.writerow((date, title, content))